## Ingest races.csv files

In [0]:
display(dbutils.fs.mounts())

In [0]:
display(dbutils.fs.ls("/mnt/formulaf1adls/raw/"))

In [0]:
races_df = spark.read.options(header='True').csv("dbfs:/mnt/formulaf1adls/raw/races.csv")
races_df.limit(10).display()

**Step 1 : using structfield and struct type create a schema**

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, TimestampType

races_schema = StructType(fields = [StructField("raceId", IntegerType(), False),
                                    StructField("year", IntegerType(), True),
                                    StructField("round", IntegerType(), True),
                                    StructField("circuitId", IntegerType(), True),
                                    StructField("name", StringType(), True),
                                    StructField("date", DateType(), True),
                                    StructField("time", StringType(), True),
                                    StructField("url", StringType(), True)
                                    ])

In [0]:
races_df = spark.read \
    .option("header", True) \
    .schema(races_schema) \
    .csv("dbfs:/mnt/formulaf1adls/raw/races.csv")

races_df.limit(5).display()

In [0]:
display(races_df.printSchema())

**Step 2: Select only the required columns and add timestamp to race timestamp and ingestion timestamp**

In [0]:
from pyspark.sql.functions import current_timestamp, to_timestamp, col, lit, concat

races_with_timestamp_df = races_df.withColumn("race_timestamp",
                                              to_timestamp(concat(col("date"), lit(" "), col("time")), 'yyyy-MM-dd HH:mm:ss')) \
                                  .withColumn("ingestion_date", current_timestamp())

In [0]:
races_with_timestamp_df.limit(10).display()

In [0]:
races_selected_df = races_with_timestamp_df.select(col("raceId").alias("race_id"),
                                                   col('year').alias('race_year'),
                                                   col('round'),
                                                   col('circuitId').alias('circuit_id'),
                                                   col('name'), 
                                                   col('ingestion_date'),
                                                   col('race_timestamp'))

races_selected_df.limit(5).display()

**Step 4: Write the df in parquet format and partitioned it on race_year**

In [0]:
races_selected_df.write.mode('overwrite').partitionBy('race_year').parquet('/mnt/formulaf1adls/processed/races')